In [ ]:
for city in ['B', 'C', 'D']:
    data = pd.read_csv(f'con2_triplegs_{city}.csv')
    
    # Due to the scale of the data, limit the frequency mining to only the first month's data (30 days) to manage computational demands effectively.
    data['finished_at'] = pd.to_datetime(data['finished_at'])
    day31 = pd.date_range("1970-01-31", periods=1, freq="h")
    day31 = pd.to_datetime(day31, utc = True)
    data = data[data['finished_at']<day31[0]]

    # add rest of code here when finalised

In [1]:
import pandas as pd
import numpy as np
from shapely import wkt
import datetime as dt
import time

In [2]:
data = pd.read_csv('triplegs_D.csv')

data['finished_at'] = pd.to_datetime(data['finished_at'])
day31 = pd.date_range("1970-01-31", periods=1, freq="h")
day31 = pd.to_datetime(day31, utc = True)
data = data[data['finished_at']<day31[0]]

data['geom'] = data['geom'].apply(wkt.loads)
data['sequences'] = data['geom'].apply(lambda x: np.round(np.array(list(x.coords))))
data = data.reset_index(drop=True)

In [3]:
data.shape

(382178, 6)

In [4]:
# DELETE after algo confirmed
subset = 1000

In [5]:
# use array to record subsequence counts with format [element 1, element 2, ..element n, support, [support_indexes]]
freq_arr = []
sequence_count = len(data.iloc[:subset])
min_sup = sequence_count * 0.02
items = set()
sequences = []

for i in range(sequence_count):
    part = list(map(tuple, data.iloc[:subset]['sequences'].iloc[i]))
    sequences.append(part)

for i in range(sequence_count):
    for j in range(len(sequences[i])):
        items.add(tuple(sequences[i][j]))

In [6]:
# count occurrences of candidate 1-sequences
for item in items:
    support = 0
    support_indexes = []
    for i in range(sequence_count):
        if item in sequences[i]:
            support+=1
            support_indexes.append(i)
    if support >= min_sup:
        freq_arr.append([item, support, support_indexes])

print(pd.DataFrame(freq_arr, columns = ['itemset', 'support', 'support_indexes']).sort_values('support', ascending = False))

items = set()

           itemset  support                                    support_indexes
15  (112.0, 133.0)       97  [512, 514, 516, 517, 518, 519, 520, 521, 523, ...
19    (191.0, 1.0)       85  [185, 187, 188, 189, 190, 191, 192, 193, 194, ...
8     (190.0, 2.0)       80  [185, 186, 187, 188, 189, 190, 191, 192, 194, ...
20  (112.0, 131.0)       63  [517, 518, 519, 520, 521, 522, 523, 524, 525, ...
5    (121.0, 98.0)       58  [383, 384, 385, 386, 387, 388, 391, 392, 393, ...
16     (32.0, 5.0)       44  [35, 36, 38, 39, 40, 45, 48, 50, 52, 53, 56, 5...
18  (116.0, 102.0)       44  [384, 388, 389, 390, 391, 392, 395, 396, 397, ...
14  (113.0, 135.0)       42  [549, 560, 923, 924, 925, 927, 928, 929, 931, ...
23  (100.0, 111.0)       38  [141, 750, 751, 752, 753, 761, 762, 763, 764, ...
2    (123.0, 55.0)       36  [816, 818, 819, 822, 823, 825, 826, 827, 828, ...
6    (159.0, 52.0)       35  [673, 675, 681, 682, 683, 687, 688, 689, 690, ...
7   (115.0, 134.0)       32  [563, 574, 927, 929, 93

In [7]:
def isSubseq(seq, sub):
    n = len(seq)
    m = len(sub)
    j = 0

    for i in range(n):
        if seq[i] == sub[j]:
            j+=1
        if j == m:
            return True
    return False

In [8]:
# generate candidate 2-sequences
for item_arr in freq_arr:
    for i in item_arr[2]:
        n = len(sequences[i])
        for j in range(n-1):
            if sequences[i][j] == item_arr[0]:
                for k in range(j+1, n):
                    items.add((item_arr[0],sequences[i][k]))
                break

In [9]:
# count occurrences of candidate 2-sequences
for item in items:
    support = 0
    support_indexes = []
    for i in range(sequence_count):
        if isSubseq(sequences[i], item):
            support+=1
            support_indexes.append(i)
    if support >= min_sup:
        freq_arr.append([item, support, support_indexes])

print(pd.DataFrame(freq_arr, columns = ['itemset', 'support', 'support_indexes']).sort_values('support', ascending = False))
        
items = set() # reset sequences set

                             itemset  support  \
15                    (112.0, 133.0)       97   
19                      (191.0, 1.0)       85   
8                       (190.0, 2.0)       80   
20                    (112.0, 131.0)       63   
38      ((190.0, 2.0), (191.0, 1.0))       62   
5                      (121.0, 98.0)       58   
27  ((112.0, 131.0), (112.0, 133.0))       50   
34      ((191.0, 1.0), (190.0, 2.0))       48   
36      ((191.0, 1.0), (191.0, 1.0))       45   
26  ((112.0, 133.0), (112.0, 133.0))       44   
16                       (32.0, 5.0)       44   
18                    (116.0, 102.0)       44   
14                    (113.0, 135.0)       42   
23                    (100.0, 111.0)       38   
2                      (123.0, 55.0)       36   
32      ((190.0, 2.0), (190.0, 2.0))       35   
6                      (159.0, 52.0)       35   
7                     (115.0, 134.0)       32   
28  ((112.0, 133.0), (112.0, 131.0))       32   
0                   